In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Constants
NUM_CLASSES = 10
NUM_TRAIN_SAMPLES_PER_CLASS = 1000
NUM_TOTAL_TRAIN_SAMPLES = NUM_CLASSES * NUM_TRAIN_SAMPLES_PER_CLASS
BATCH_SIZE = 64
EPOCHS = 100
LEARNING_RATE = 0.001

In [3]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # CIFAR-10 normalization
])
# Load CIFAR10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=train_transform)

# Select a subset of training data
indices = torch.arange(NUM_TOTAL_TRAIN_SAMPLES)
subset_indices = torch.cat([indices[i:i+NUM_TRAIN_SAMPLES_PER_CLASS] for i in range(0, len(indices), NUM_TRAIN_SAMPLES_PER_CLASS)])
train_dataset = torch.utils.data.Subset(train_dataset, subset_indices)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Define ResNet20 model
class ResNet20(nn.Module):
  def __init__(self, num_classes=10):
    super(ResNet20, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    self.layer1 = self._make_layer(16, 16, 3)
    self.layer2 = self._make_layer(16, 32, 3, stride=2)
    self.layer3 = self._make_layer(32, 64, 3, stride=2)
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(64, num_classes)

  def _make_layer(self, in_channels, out_channels, num_blocks, stride=1):
    layers = []
    layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
    layers.append(nn.BatchNorm2d(out_channels))
    layers.append(nn.ReLU(inplace=True))

    for _ in range(1, num_blocks):
        layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))
        layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU(inplace=True))

    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)

    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    x = self.fc(x)

    return x

In [5]:
 model = ResNet20(num_classes=NUM_CLASSES)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
  _, predicted = torch.max(outputs.data, 1)
  correct = (predicted == labels).sum().item()
  total = labels.size(0)
  accuracy = correct / total
  return accuracy

# Training loop
train_loss_history = []
train_acc_history = []
test_acc_history = []

for epoch in range(EPOCHS):
  model.train()  # Set model to training mode
  train_loss = 0.0
  train_correct = 0
  train_total = 0

  # Iterate over training dataset
  for inputs, labels in train_loader:
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_loss += loss.item()
      train_correct += (outputs.argmax(1) == labels).sum().item()
      train_total += labels.size(0)

  train_loss /= len(train_loader)
  train_accuracy = train_correct / train_total

  # Evaluate on the test dataset
  model.eval()  # Set model to evaluation mode
  test_correct = 0
  test_total = 0

  with torch.no_grad():
      for inputs, labels in test_loader:
          outputs = model(inputs)
          test_correct += (outputs.argmax(1) == labels).sum().item()
          test_total += labels.size(0)

  test_accuracy = test_correct / test_total

  # Record history
  train_loss_history.append(train_loss)
  train_acc_history.append(train_accuracy)
  test_acc_history.append(test_accuracy)

  print(f"Epoch {epoch+1}/{EPOCHS} - "
    f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
    f"Test Acc: {test_accuracy:.4f}")

Epoch 1/100 - Train Loss: 1.7627, Train Acc: 0.3486, Test Acc: 0.3945
Epoch 2/100 - Train Loss: 1.4534, Train Acc: 0.4684, Test Acc: 0.4697
Epoch 3/100 - Train Loss: 1.3175, Train Acc: 0.5207, Test Acc: 0.5150
Epoch 4/100 - Train Loss: 1.2080, Train Acc: 0.5587, Test Acc: 0.4989
Epoch 5/100 - Train Loss: 1.1143, Train Acc: 0.6054, Test Acc: 0.5313
Epoch 6/100 - Train Loss: 1.0337, Train Acc: 0.6301, Test Acc: 0.5838
Epoch 7/100 - Train Loss: 0.9471, Train Acc: 0.6669, Test Acc: 0.6012
Epoch 8/100 - Train Loss: 0.8712, Train Acc: 0.6860, Test Acc: 0.6048
Epoch 9/100 - Train Loss: 0.8066, Train Acc: 0.7110, Test Acc: 0.6145
Epoch 10/100 - Train Loss: 0.7276, Train Acc: 0.7438, Test Acc: 0.6080
Epoch 11/100 - Train Loss: 0.6687, Train Acc: 0.7645, Test Acc: 0.5801
Epoch 12/100 - Train Loss: 0.5955, Train Acc: 0.7886, Test Acc: 0.6219
Epoch 13/100 - Train Loss: 0.5340, Train Acc: 0.8116, Test Acc: 0.5650
Epoch 14/100 - Train Loss: 0.4554, Train Acc: 0.8399, Test Acc: 0.6187
Epoch 15/100 - 

In [6]:
model.eval()  # Set model to evaluation mode
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        test_correct += (outputs.argmax(1) == labels).sum().item()
        test_total += labels.size(0)

test_accuracy = test_correct / test_total



In [7]:
print("Final Test Accuracy:",test_accuracy)

Final Test Accuracy: 0.6266
